In [119]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec  19 14:44:06 2019

@author: Mohamed.Imran
"""

import os
import pandas as pd
import numpy as np
import datetime as dt
from sklearn import linear_model
#from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from prophet import Prophet
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#os.chdir(r"C:\Users\moham\OneDrive - University of Central Missouri\Desktop\Imran\Training\Inceptez\Content\Data-Science-Central-Online-master\Data-Science-Central-Online-master\Time Series")
#pwd = os.getcwd()

data = pd.read_excel("TS_Data.xlsx") #Read your decomposition file here
data = data.tail(-1)
#Regression model
def Regression(data):
    data["Date"] = pd.to_datetime(data["Date"])
    data=data.sort_values(["Date"])
    data["Quarter"] = data["Date"].dt.quarter
    data["Year"] = data["Date"].dt.year
    data["Month"] = data["Date"].dt.month
    
    Train=data[(data.Year>=2015)&(data.Year<2019)] #modify date according to your dataset; Train : 2017-2018
    Test=data[(data.Year==2019)]  #modify date according to your dataset; Test : 2019
    
    
    Train["SI_Y"]=Train["Volume"]/Train.groupby("Year")["Volume"].transform(np.mean)
    Train["F_SI"]=Train.groupby("Month")["SI_Y"].transform(np.mean)
    Train["D_Seasonalised_trend"] = Train["Volume"]/Train["F_SI"]    
    Train["Level_index1"]=np.mean(Train[(Train.Year==2018)&(Train.Quarter==1)]["D_Seasonalised_trend"])/np.mean(Train[(Train.Year==2017)&(Train.Quarter==4)]["D_Seasonalised_trend"])
    
    numer1=np.mean(Train[(Train.Year==2018)&(Train.Quarter==3)]["D_Seasonalised_trend"])/np.mean(Train[(Train.Year==2018)&(Train.Quarter==2)]["D_Seasonalised_trend"])
    numer2=np.mean(Train[(Train.Year==2018)&(Train.Quarter==4)]["D_Seasonalised_trend"])/np.mean(Train[(Train.Year==2018)&(Train.Quarter==3)]["D_Seasonalised_trend"])
    
    
    Train["Level_index2"]=np.mean([numer1,numer2])
    Train=Train.sort_values(["Date"])
    Train.index=range(len(Train))
    Train["ID"]=range(1,(len(Train)+1))
    
    Train["Deleveled_series"]=np.where(Train.Year==2017, Train["D_Seasonalised_trend"]*Train["Level_index1"],Train["D_Seasonalised_trend"])
    
    lm = linear_model.LinearRegression()
    X = np.array(Train[["ID", "Marketing spends"]]) # In case of no extra variable in the dataset, remove the extra variable name from the list, then append the line with ".reshape(-1, 1)"
    Y = np.array(Train["Deleveled_series"]).reshape(-1,1)
    
    model = lm.fit(X,Y)
    
    Test["ID"]=range(len(Test))
    Test["ID"]=Test["ID"]+max(Train["ID"])
    X_test=np.array(Test[["ID", "Marketing spends"]]) # In case of no extra variable in the dataset, remove the extra variable name from the list, then append the line with ".reshape(-1, 1)"
    Y_test=model.predict(X_test)
    
    Pred1 = Y_test*Train.iloc[0]["Level_index2"]*np.array(Train.iloc[0:len(Y_test)]["F_SI"]).reshape(-1,1)
    Test["Predictions"]=Pred1
    
    return(Test['Predictions'])

#Arima model
def Arima(data): 
    X = data['Volume'].values
    size = np.sum(data['Date']<='12/31/2018')
    train, test = X[0:size], X[size:len(X)]
    history = [x for x in train]
    predictions = list()  
        
    for t in range(len(test)):
    	model = ARIMA(history, order=(1,1,0))
    	model_fit = model.fit()
    	output = model_fit.forecast()
    	yhat = output[0]
    	predictions.append(yhat)
    	obs = test[t]
    	history.append(obs)
    return predictions   

#Holts-Winter model
def Holts_winter(data):
    inter_df = data[['Volume']]
    size = np.sum(data['Date']<='12/31/2018')
    train, test = inter_df.iloc[:size, 0], inter_df.iloc[size:, 0]
    model = ExponentialSmoothing(np.array(train), seasonal='mul', seasonal_periods=12).fit()
    pred = model.predict(start=test.index[0], end=test.index[-1])
    return pd.DataFrame(pred, columns=['yhat'])

#Fbprophet
def Fbprophet(data):
    size = np.sum(data['ds']<='12/31/2018')
    inter_df = data.iloc[:size, :]
    m = Prophet(weekly_seasonality=False, daily_seasonality=False)
    m.fit(inter_df)
    future = m.make_future_dataframe(periods=12, freq='M')
    forecast = m.predict(future)
    fcst = forecast['yhat'].tail(12)
    return fcst

#Simple Exponential Smoothing model
def Ses(data):
    inter_df = data[['Volume']]
    size = np.sum(data['Date']<='12/31/2018')
    train, test = inter_df.iloc[:size, 0], inter_df.iloc[size:, 0]
    model = SimpleExpSmoothing(np.array(train)).fit()
    pred = model.predict(start=test.index[0], end=test.index[-1])
    return pd.DataFrame(pred, columns=['yhat'])



def Regression_2lag(data):
    data["Variable_1"] = data["Marketing spends"].shift(2)
    data = data.loc[2:, :]
    return Regression(data)


required_cols = [col for col in data.columns if col not in ['Date', 'Marketing spends']]


Result=pd.DataFrame()

for model in [Regression, Arima, Holts_winter, Ses, Fbprophet, Regression_2lag]:
    for i in required_cols:
        data['Date'] = pd.to_datetime(data['Date'])
        to_func = data[["Date", "Marketing spends", i]]
        to_func.columns=["Date","Marketing spends", "Volume"]
        print(to_func.head())
        if model == Fbprophet:
            to_func.columns=["ds","Marketing spends", "y"]
            Result_inter = model(to_func[['ds', 'y']])
            print(Result_inter)
            Result_inter.name = model.__name__ + "_" +  i
            Result_inter.index = range(len(Result_inter))
            Result = pd.concat([Result, Result_inter], axis = 1)
        elif model == Arima:
            Result_inter = model(to_func)
            Result_inter = pd.DataFrame(Result_inter, columns = ["ARIMA_" + i])
            Result_inter.index=range(len(Result_inter))
            Result = pd.concat([Result, Result_inter], axis = 1)
        else:
            Result_inter = model(to_func)
            Result_inter.name = model.__name__ + "_" +  i
            Result_inter.index=range(len(Result_inter))
            Result = pd.concat([Result, Result_inter], axis = 1)



Result.to_csv('Forecast.csv')



        Date Marketing spends   Volume
1 2017-01-01           176732  1942.21
2 2017-02-01           180486  1749.93
3 2017-03-01           180455  2399.42
4 2017-04-01           185070  2126.85
5 2017-05-01           195915   2242.5
        Date Marketing spends   Volume
1 2017-01-01           176732  1942.21
2 2017-02-01           180486  1749.93
3 2017-03-01           180455  2399.42
4 2017-04-01           185070  2126.85
5 2017-05-01           195915   2242.5
        Date Marketing spends   Volume
1 2017-01-01           176732  1942.21
2 2017-02-01           180486  1749.93
3 2017-03-01           180455  2399.42
4 2017-04-01           185070  2126.85
5 2017-05-01           195915   2242.5
        Date Marketing spends   Volume
1 2017-01-01           176732  1942.21
2 2017-02-01           180486  1749.93
3 2017-03-01           180455  2399.42
4 2017-04-01           185070  2126.85
5 2017-05-01           195915   2242.5


02:14:38 - cmdstanpy - INFO - Chain [1] start processing


        Date Marketing spends   Volume
1 2017-01-01           176732  1942.21
2 2017-02-01           180486  1749.93
3 2017-03-01           180455  2399.42
4 2017-04-01           185070  2126.85
5 2017-05-01           195915   2242.5


02:14:39 - cmdstanpy - INFO - Chain [1] done processing


24    1648.851857
25    1624.350486
26    1602.220215
27    1577.718844
28    1554.007840
29    1529.506468
30    1505.795464
31    1481.294093
32    1456.792721
33    1433.081717
34    1408.580346
35    1384.869341
Name: yhat, dtype: float64
        Date Marketing spends   Volume
1 2017-01-01           176732  1942.21
2 2017-02-01           180486  1749.93
3 2017-03-01           180455  2399.42
4 2017-04-01           185070  2126.85
5 2017-05-01           195915   2242.5


In [120]:
Result

,Regression_Unnamed: 2,ARIMA_Unnamed: 2,yhat,yhat,Fbprophet_Unnamed: 2,Regression_2lag_Unnamed: 2
0,1768.788713,1303.810064,1473.399434,1739.648643,1648.851857,1655.679462
1,1644.867440,1794.298500,1714.389978,1739.648643,1624.350486,1899.581115
2,1879.092523,1970.266340,1622.873362,1739.648643,1602.220215,1786.388650
3,1768.116498,1909.750770,1718.502455,1739.648643,1577.718844,1876.394032
4,1860.779308,1892.420707,1705.690392,1739.648643,1554.007840,1798.965371
5,1783.719718,1955.382202,1625.238339,1739.648643,1529.506468,1760.209644
6,1737.506669,1741.998443,1408.868509,1739.648643,1505.795464,1490.943996
7,1467.594941,1860.362031,1775.133142,1739.648643,1481.294093,1882.785277
8,1858.102359,1697.693296,2197.688795,1739.648643,1456.792721,2284.272561
9,2261.536173,1921.679863,1829.755484,1739.648643,1433.081717,1886.430392
